# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [1]:
#Your code here

import pandas as pd
import numpy as np
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style('darkgrid')
%matplotlib inline

df = pd.read_csv('homepage_actions.csv')

In [2]:
df.head()

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8188 entries, 0 to 8187
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   timestamp  8188 non-null   object
 1   id         8188 non-null   int64 
 2   group      8188 non-null   object
 3   action     8188 non-null   object
dtypes: int64(1), object(3)
memory usage: 256.0+ KB


In [4]:
len(set(df['id'])) # only 6,300 unique ids, so there are some repeated customers

6328

In [5]:
display(df['group'].value_counts())
display(df.action.value_counts())
df.groupby(['group', 'action']).agg('count')

control       4264
experiment    3924
Name: group, dtype: int64

view     6328
click    1860
Name: action, dtype: int64

timestamp    id
group      action                 
control    click         932   932
           view         3332  3332
experiment click         928   928
           view         2996  2996

In [6]:
# did any users both click and view?

cids = set(df[df.action=='click']['id'])
vids = set(df[df.action=='view']['id'])
print("Number of viewers: {} \tNumber of clickers: {}".format(len(vids), len(cids)))

# subtracting two sets results in difference between the sets
print("Number of Viewers who didn't click: {}".format(len(vids-cids))) 
print("Number of Clickers who didn't view: {}".format(len(cids-vids)))

Number of viewers: 6328 	Number of clickers: 1860
Number of Viewers who didn't click: 4468
Number of Clickers who didn't view: 0


In [7]:
# are any users in both control and experiment?
ctrlids = set(df[df.group=='control']['id'])
expids = set(df[df.group=='experiment']['id'])

print("Overlap of Exp and Control Groups: {}".format(len(ctrlids&expids))) 

Overlap of Exp and Control Groups: 0


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [2]:
#Your code here

df['action'][0]
clicked = [1 if row=='click' else 0 for row in df['action']]
viewed = [1 if row=='view' else 0 for row in df['action']]

df['click'] = clicked
df['view'] = viewed
df.head()

,timestamp,id,group,action,click,view
0,2016-09-24 17:42:27.839496,804196,experiment,view,0,1
1,2016-09-24 19:19:03.542569,434745,experiment,view,0,1
2,2016-09-24 19:36:00.944135,507599,experiment,view,0,1
3,2016-09-24 19:59:02.646620,671993,control,view,0,1
4,2016-09-24 20:26:14.466886,536734,experiment,view,0,1


In [3]:
print(sum(df['click'][df['group']=='experiment']))
print(sum(df['click'][df['group']=='control']))
print(sum(df['view'][df['group']=='experiment']))
print(sum(df['view'][df['group']=='control']))

928
932
2996
3332


In [4]:
control = df[['id', 'click', 'view']][df['group']=='control']
exp = df[['id', 'click', 'view']][df['group']=='experiment']

#control.drop_duplicates()
#exp.drop_duplicates()

control = pd.pivot_table(control, index='id', aggfunc='sum')
exp = pd.pivot_table(exp, index='id', aggfunc='sum')

display(control.head())
display(exp.head())

#control = [sum(df['click'][df.group=='control']),sum(df['view'][df.group=='control'])]
#exp = [sum(df['click'][df.group=='experiment']),sum(df['view'][df.group=='experiment'])]

print(len(control))
print(len(exp))

sum(control['view'])

,click,view
id,,
182994,1,1
183089,0,1
183248,1,1
183515,0,1
183524,0,1


,click,view
id,,
182988,0,1
183136,0,1
183141,1,1
183283,0,1
183389,0,1


3332
2996


3332

In [5]:
#import flatiron_stats as fs
#fs.p_value_welch_ttest(control.click, experiment.click)

t, p = stats.ttest_ind(control.click, exp.click, equal_var = False)
p / 2

# p is less than 0.01, hence significant different between groups

0.004466402814337101

In [70]:
#df['count'] = 1
#df[df.group=='control'].pivot(index='id', columns='action', values='count').fillna(value=0)

#len(df[df.group=='control'].pivot(index='id', columns='action', values='count').fillna(value=0))

3332

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [6]:
#Your code here

ctrl_clk_rate = sum(df['click'][df['group']=='control']) / len(set(df['id'][df['group']=='control']))
ctrl_clk_rate

exp_clk = ctrl_clk_rate * len(set(df['id'][df['group']=='experiment']))
exp_clk

838.0168067226891

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [9]:
#Your code here

# binomial distribution: mu = n*p, var = n*p*(1-p) where p is probability of success

n = len(exp)
p = ctrl_clk_rate
var = n * p * (1-p)
std = np.sqrt(var)
print(std)

z = (sum(df['click'][df['group']=='experiment']) - exp_clk) / std
z

24.568547907005815


3.6625360854823588

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [11]:
#Your code here
stats.norm.sf(abs(z))

0.00012486528006951198

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: P value is lower but both result in rejection of the null hypothesis.

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.